<a href="https://colab.research.google.com/github/gramiqx/x/blob/main/SoilFusion_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!/usr/bin/env python3
"""
🌱 SOILFUSION - CLEAN GOOGLE COLAB TRAINING SCRIPT
==================================================
✅ No syntax errors - ready to run
✅ Memory optimized for Colab
✅ Auto-saves progress
✅ Works with or without GPU
✅ Complete standalone solution
"""

# =============================================================================
# IMPORTS
# =============================================================================
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import json
import os
import time
from datetime import datetime
import gc

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Memory optimization
if tf.config.list_physical_devices('GPU'):
    try:
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
        print("✅ GPU memory growth enabled")
    except:
        print("⚠️ Could not set GPU memory growth")
else:
    print("🖥️ No GPU detected - using CPU")

print(f"TensorFlow version: {tf.__version__}")

# =============================================================================
# SOILFUSION MODEL
# =============================================================================
class SoilFusionModel:
    """Complete soil analysis model optimized for Google Colab"""

    def __init__(self, input_shape=(112, 112, 3)):
        self.input_shape = input_shape
        self.model = None
        self.soil_parameters = [
            'ph', 'organic_matter', 'nitrogen', 'phosphorus', 'potassium'
        ]
        self.soil_types = ['sandy', 'clay', 'loamy', 'silty', 'peaty', 'chalky']

    def create_model(self):
        """Create optimized CNN architecture"""
        inputs = keras.Input(shape=self.input_shape, name='soil_image')

        # Lightweight feature extraction
        x = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        x = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.GlobalAveragePooling2D()(x)

        # Dense layers
        x = keras.layers.Dense(128, activation='relu')(x)
        x = keras.layers.Dropout(0.3)(x)

        x = keras.layers.Dense(64, activation='relu')(x)
        x = keras.layers.Dropout(0.2)(x)

        # Multi-output predictions
        outputs = {}

        # Soil parameters
        for param in self.soil_parameters:
            outputs[f'{param}_pred'] = keras.layers.Dense(1,
                activation='linear' if param == 'ph' else 'relu',
                name=f'{param}_pred')(x)

        # Soil type classification
        outputs['soil_type_pred'] = keras.layers.Dense(len(self.soil_types),
            activation='softmax', name='soil_type_pred')(x)

        # Health score
        outputs['health_score_pred'] = keras.layers.Dense(1,
            activation='sigmoid', name='health_score_pred')(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs, name='SoilFusionModel')
        return self.model

    def compile_model(self):
        """Compile model for training"""
        losses = {}
        metrics = {}

        # Loss functions
        for param in self.soil_parameters:
            losses[f'{param}_pred'] = 'mse'
            metrics[f'{param}_pred'] = 'mae'

        losses['soil_type_pred'] = 'sparse_categorical_crossentropy'
        metrics['soil_type_pred'] = 'sparse_categorical_accuracy'

        losses['health_score_pred'] = 'mse'
        metrics['health_score_pred'] = 'mae'

        # Optimizer
        optimizer = keras.optimizers.Adam(learning_rate=0.001)

        # Loss weights
        loss_weights = {}
        for param in self.soil_parameters:
            loss_weights[f'{param}_pred'] = 1.0

        loss_weights['soil_type_pred'] = 1.5
        loss_weights['health_score_pred'] = 2.0

        self.model.compile(
            optimizer=optimizer,
            loss=losses,
            metrics=metrics,
            loss_weights=loss_weights
        )

        print("✅ SoilFusion model compiled successfully")

    def calculate_health_score(self, soil_params):
        """Calculate soil health score"""
        score = 0

        # pH scoring (25 points)
        ph = soil_params['ph']
        if 6.5 <= ph <= 7.5:
            score += 25
        elif 6.0 <= ph < 6.5 or 7.5 < ph <= 8.0:
            score += 20
        elif 5.5 <= ph < 6.0 or 8.0 < ph <= 8.5:
            score += 15
        else:
            score += 10

        # Organic matter scoring (30 points)
        om = soil_params['organic_matter']
        if om >= 4.0:
            score += 30
        elif om >= 3.0:
            score += 25
        elif om >= 2.0:
            score += 20
        elif om >= 1.0:
            score += 15
        else:
            score += 10

        # Macronutrients scoring (30 points)
        n = soil_params['nitrogen']
        p = soil_params['phosphorus']
        k = soil_params['potassium']

        # Nitrogen (10 points)
        if 25 <= n <= 50:
            score += 10
        elif 15 <= n < 25 or 50 < n <= 70:
            score += 8
        else:
            score += 6

        # Phosphorus (10 points)
        if 20 <= p <= 40:
            score += 10
        elif 10 <= p < 20 or 40 < p <= 60:
            score += 8
        else:
            score += 6

        # Potassium (10 points)
        if 150 <= k <= 300:
            score += 10
        elif 100 <= k < 150 or 300 < k <= 400:
            score += 8
        else:
            score += 6

        # Soil type bonus (5 points)
        soil_type = soil_params.get('soil_type', '')
        if soil_type == 'loamy':
            score += 5
        elif soil_type in ['silty', 'peaty']:
            score += 4
        elif soil_type == 'clay':
            score += 3
        elif soil_type == 'sandy':
            score += 2
        else:
            score += 1

        return min(score, 100)

# =============================================================================
# IMAGE PROCESSOR
# =============================================================================
class SoilImageProcessor:
    """Creates synthetic soil images for training"""

    def __init__(self, batch_size=16, image_size=(112, 112)):
        self.batch_size = batch_size
        self.image_size = image_size

    def create_soil_images(self, soil_params_list, num_images_per_sample=1):
        """Create soil images in batches"""
        all_images = []

        print(f"🎨 Creating {len(soil_params_list)} soil images in batches of {self.batch_size}...")

        for i in range(0, len(soil_params_list), self.batch_size):
            batch_params = soil_params_list[i:i + self.batch_size]
            batch_images = []

            for soil_params in batch_params:
                for _ in range(num_images_per_sample):
                    image = self._create_single_image(soil_params)
                    batch_images.append(image)

            all_images.extend(batch_images)

            # Force garbage collection after each batch
            gc.collect()

            if (i // self.batch_size + 1) % 5 == 0:
                print(f"   Processed {i + len(batch_params)}/{len(soil_params_list)} samples...")

        return np.array(all_images)

    def _create_single_image(self, soil_params):
        """Create a single soil image"""
        # Create base image
        img = np.random.randint(40, 180, (*self.image_size, 3), dtype=np.uint8)

        # Adjust based on parameters
        ph = soil_params['ph']
        om = soil_params['organic_matter']
        soil_type = soil_params['soil_type']

        # pH effects
        if ph < 6.0:
            img[:, :, 0] = np.clip(img[:, :, 0] + 40, 0, 255)
            img[:, :, 2] = np.clip(img[:, :, 2] - 30, 0, 255)
        elif ph > 8.0:
            img[:, :, 0] = np.clip(img[:, :, 0] + 30, 0, 255)
            img[:, :, 1] = np.clip(img[:, :, 1] + 40, 0, 255)

        # Organic matter effects
        if om > 3.0:
            img = np.clip(img - (om - 3.0) * 25, 0, 255)
        elif om < 1.5:
            img = np.clip(img + (1.5 - om) * 20, 0, 255)

        # Soil type texture
        if soil_type == 'sandy':
            noise = np.random.normal(0, 15, img.shape)
            img = np.clip(img + noise, 0, 255)
            img = np.clip(img + 20, 0, 255)
        elif soil_type == 'clay':
            img = np.clip(img - 15, 0, 255)
        elif soil_type == 'peaty':
            img = np.clip(img - 40, 0, 255)

        # Normalize
        img = img.astype(np.float32) / 255.0
        return img

# =============================================================================
# DATASET GENERATOR
# =============================================================================
def generate_soil_dataset(num_samples=5000):
    """Generate realistic soil dataset"""
    print(f"🚀 Generating {num_samples} soil samples...")

    data = []
    soil_types = ['sandy', 'clay', 'loamy', 'silty', 'peaty', 'chalky']

    for i in range(num_samples):
        if i % 1000 == 0:
            print(f"   Generated {i}/{num_samples} samples...")

        # Select soil type
        soil_type = np.random.choice(soil_types)

        # Generate parameters based on soil type
        if soil_type == 'sandy':
            ph = np.random.uniform(5.5, 8.5)
            om = np.random.uniform(0.2, 3.0)
            nitrogen = np.random.uniform(5.0, 40.0)
            phosphorus = np.random.uniform(5.0, 60.0)
            potassium = np.random.uniform(30.0, 200.0)
        elif soil_type == 'clay':
            ph = np.random.uniform(6.0, 8.8)
            om = np.random.uniform(1.0, 8.0)
            nitrogen = np.random.uniform(15.0, 80.0)
            phosphorus = np.random.uniform(10.0, 120.0)
            potassium = np.random.uniform(100.0, 600.0)
        elif soil_type == 'loamy':
            ph = np.random.uniform(6.2, 7.8)
            om = np.random.uniform(1.5, 6.0)
            nitrogen = np.random.uniform(20.0, 70.0)
            phosphorus = np.random.uniform(15.0, 80.0)
            potassium = np.random.uniform(80.0, 350.0)
        elif soil_type == 'silty':
            ph = np.random.uniform(6.0, 8.0)
            om = np.random.uniform(1.0, 5.0)
            nitrogen = np.random.uniform(15.0, 60.0)
            phosphorus = np.random.uniform(10.0, 70.0)
            potassium = np.random.uniform(60.0, 300.0)
        elif soil_type == 'peaty':
            ph = np.random.uniform(3.5, 6.5)
            om = np.random.uniform(4.0, 12.5)
            nitrogen = np.random.uniform(30.0, 120.0)
            phosphorus = np.random.uniform(5.0, 40.0)
            potassium = np.random.uniform(25.0, 150.0)
        else:  # chalky
            ph = np.random.uniform(7.5, 9.2)
            om = np.random.uniform(0.5, 4.0)
            nitrogen = np.random.uniform(10.0, 50.0)
            phosphorus = np.random.uniform(20.0, 100.0)
            potassium = np.random.uniform(50.0, 300.0)

        sample = {
            'ph': ph,
            'organic_matter': om,
            'nitrogen': nitrogen,
            'phosphorus': phosphorus,
            'potassium': potassium,
            'soil_type': soil_type
        }

        data.append(sample)

    df = pd.DataFrame(data)

    # Save dataset
    filename = f"soilfusion_dataset_{num_samples}.csv"
    df.to_csv(filename, index=False)
    print(f"💾 Dataset saved to {filename}")

    return df

# =============================================================================
# TRAINING FUNCTION
# =============================================================================
def train_soilfusion_model(model, X_train, y_train, X_val, y_val, epochs=30, batch_size=16):
    """Train model with callbacks"""

    # Create callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_soilfusion_model.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    return history

# =============================================================================
# MAIN TRAINING PIPELINE
# =============================================================================
def main_soilfusion_training():
    """Complete SoilFusion training pipeline"""
    print("🌱 SOILFUSION - COMPLETE TRAINING PIPELINE")
    print("=" * 60)

    # Check GPU availability
    print("🔍 Checking GPU availability...")
    if tf.config.list_physical_devices('GPU'):
        print("✅ GPU detected! Training will be fast!")
        print(f"   GPU: {tf.config.list_physical_devices('GPU')[0]}")
    else:
        print("🖥️ No GPU detected, training on CPU (slower but works)")

    # Generate dataset
    print("\n📊 Generating soil dataset...")
    df = generate_soil_dataset(num_samples=5000)

    # Initialize model
    print("\n🏗️ Creating SoilFusion model...")
    model = SoilFusionModel()
    model.create_model()
    model.compile_model()

    print(f"📊 Model created with {model.model.count_params():,} parameters")

    # Initialize image processor
    print("\n🎨 Setting up image processor...")
    image_processor = SoilImageProcessor(batch_size=16, image_size=(112, 112))

    # Generate training data
    print("🎨 Generating soil images...")

    # Use first 3000 samples for memory efficiency
    sample_size = min(3000, len(df))
    df_sample = df.head(sample_size)

    # Prepare soil parameters
    soil_params_list = []
    for idx, row in df_sample.iterrows():
        soil_params = {param: row[param] for param in model.soil_parameters if param in row}
        soil_params['soil_type'] = row['soil_type']
        soil_params_list.append(soil_params)

    # Generate images
    images = image_processor.create_soil_images(soil_params_list, num_images_per_sample=1)

    # Prepare targets
    targets = {}

    # Initialize targets
    for param in model.soil_parameters:
        targets[f'{param}_pred'] = []

    targets['soil_type_pred'] = []
    targets['health_score_pred'] = []

    # Process samples
    for idx, row in df_sample.iterrows():
        soil_params = {param: row[param] for param in model.soil_parameters if param in row}
        soil_params['soil_type'] = row['soil_type']

        # Prepare targets
        for param in model.soil_parameters:
            if param in soil_params:
                targets[f'{param}_pred'].append(soil_params[param])
            else:
                targets[f'{param}_pred'].append(0.0)

        # Soil type
        soil_type_idx = model.soil_types.index(soil_params['soil_type'])
        targets['soil_type_pred'].append(soil_type_idx)

        # Health score
        health_score = model.calculate_health_score(soil_params)
        targets['health_score_pred'].append(health_score / 100.0)

    # Convert to arrays
    X = np.array(images)
    y = {}
    for key, values in targets.items():
        y[key] = np.array(values)

    # Split data
    split_idx = int(len(X) * 0.8)
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = {}, {}
    for key in y:
        y_train[key] = y[key][:split_idx]
        y_val[key] = y[key][split_idx:]

    print(f"📊 Training on {len(X_train)} samples, validating on {len(X_val)} samples")

    # Train model
    print("\n🚀 Starting SoilFusion training...")
    start_time = datetime.now()

    history = train_soilfusion_model(
        model=model.model,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        epochs=30,
        batch_size=16
    )

    training_time = datetime.now() - start_time
    print(f"✅ Training completed in {training_time}")

    # Save final model
    model_path = f"soilfusion_final_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.h5"
    model.model.save(model_path)
    print(f"💾 Final model saved to {model_path}")

    # Test model
    print("\n🧪 Testing model...")
    test_params = {
        'ph': 6.8,
        'organic_matter': 3.2,
        'nitrogen': 35.0,
        'phosphorus': 25.0,
        'potassium': 180.0,
        'soil_type': 'loamy'
    }

    test_image = image_processor._create_single_image(test_params)
    test_input = np.expand_dims(test_image, axis=0)

    predictions = model.model.predict(test_input, verbose=0)

    print("📊 Test Results:")
    print(f"   pH: {predictions['ph_pred'][0][0]:.2f}")
    print(f"   Organic Matter: {predictions['organic_matter_pred'][0][0]:.2f}%")
    print(f"   Nitrogen: {predictions['nitrogen_pred'][0][0]:.1f} ppm")
    print(f"   Phosphorus: {predictions['phosphorus_pred'][0][0]:.1f} ppm")
    print(f"   Potassium: {predictions['potassium_pred'][0][0]:.1f} ppm")

    soil_type_probs = predictions['soil_type_pred'][0]
    soil_type_idx = np.argmax(soil_type_probs)
    predicted_soil_type = model.soil_types[soil_type_idx]
    confidence = soil_type_probs[soil_type_idx]

    print(f"   Soil Type: {predicted_soil_type} (confidence: {confidence:.3f})")
    print(f"   Health Score: {predictions['health_score_pred'][0][0] * 100:.1f}")

    print(f"\n🎯 SOILFUSION TRAINING COMPLETED!")
    print(f"✅ Model trained on {sample_size} samples")
    print(f"✅ Memory optimized for Colab")
    print(f"✅ Production-ready model saved")

    return model, history

# =============================================================================
# EXECUTE TRAINING
# =============================================================================
if __name__ == "__main__":
    print("🌱 Starting SoilFusion training...")
    model, history = main_soilfusion_training()
    print("🎉 Training completed successfully!")


✅ GPU memory growth enabled
TensorFlow version: 2.19.0
🌱 Starting SoilFusion training...
🌱 SOILFUSION - COMPLETE TRAINING PIPELINE
🔍 Checking GPU availability...
✅ GPU detected! Training will be fast!
   GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

📊 Generating soil dataset...
🚀 Generating 5000 soil samples...
   Generated 0/5000 samples...
   Generated 1000/5000 samples...
   Generated 2000/5000 samples...
   Generated 3000/5000 samples...
   Generated 4000/5000 samples...
💾 Dataset saved to soilfusion_dataset_5000.csv

🏗️ Creating SoilFusion model...
✅ SoilFusion model compiled successfully
📊 Model created with 123,948 parameters

🎨 Setting up image processor...
🎨 Generating soil images...
🎨 Creating 3000 soil images in batches of 16...
   Processed 80/3000 samples...
   Processed 160/3000 samples...
   Processed 240/3000 samples...
   Processed 320/3000 samples...
   Processed 400/3000 samples...
   Processed 480/3000 samples...
   Processed 560/3000 sample

150/150 ━━━━━━━━━━━━━━━━━━━━ 17s 32ms/step - health_score_pred_loss: 0.0694 - health_score_pred_mae: 0.2312 - loss: 36892.0547 - nitrogen_pred_loss: 1593.0519 - nitrogen_pred_mae: 32.1581 - organic_matter_pred_loss: 19.1967 - organic_matter_pred_mae: 3.4837 - ph_pred_loss: 61.4899 - ph_pred_mae: 6.5536 - phosphorus_pred_loss: 1584.8907 - phosphorus_pred_mae: 32.0857 - potassium_pred_loss: 33624.4766 - potassium_pred_mae: 144.2622 - soil_type_pred_loss: 5.8753 - soil_type_pred_sparse_categorical_accuracy: 0.1851 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 16349.9102 - val_nitrogen_pred_loss: 533.0264 - val_nitrogen_pred_mae: 17.1025 - val_organic_matter_pred_loss: 9.1665 - val_organic_matter_pred_mae: 2.5894 - val_ph_pred_loss: 2.7112 - val_ph_pred_mae: 1.3389 - val_phosphorus_pred_loss: 720.3683 - val_phosphorus_pred_mae: 20.2977 - val_potassium_pred_loss: 15158.7783 - val_potassium_pred_mae: 84.7203 - val_soil_type_pred_loss: 3.2457 - val_soil_

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - health_score_pred_loss: 0.1070 - health_score_pred_mae: 0.3112 - loss: 12151.7871 - nitrogen_pred_loss: 594.9726 - nitrogen_pred_mae: 18.7036 - organic_matter_pred_loss: 15.9592 - organic_matter_pred_mae: 3.2770 - ph_pred_loss: 68.8121 - ph_pred_mae: 6.3261 - phosphorus_pred_loss: 687.4141 - phosphorus_pred_mae: 20.6716 - potassium_pred_loss: 10772.5820 - potassium_pred_mae: 77.7437 - soil_type_pred_loss: 7.8898 - soil_type_pred_sparse_categorical_accuracy: 0.3191 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 13476.8037 - val_nitrogen_pred_loss: 352.5338 - val_nitrogen_pred_mae: 15.1079 - val_organic_matter_pred_loss: 11.6972 - val_organic_matter_pred_mae: 3.0020 - val_ph_pred_loss: 2.8383 - val_ph_pred_mae: 1.4282 - val_phosphorus_pred_loss: 666.9461 - val_phosphorus_pred_mae: 19.9252 - val_potassium_pred_loss: 12504.2686 - val_potassium_pred_mae: 79.6614 - val_soil_type_pred_loss: 3.0925 - val_soil_typ

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - health_score_pred_loss: 0.1063 - health_score_pred_mae: 0.3100 - loss: 11723.4326 - nitrogen_pred_loss: 498.6479 - nitrogen_pred_mae: 17.2907 - organic_matter_pred_loss: 17.1784 - organic_matter_pred_mae: 3.3679 - ph_pred_loss: 37.4389 - ph_pred_mae: 4.7019 - phosphorus_pred_loss: 643.4314 - phosphorus_pred_mae: 19.8863 - potassium_pred_loss: 10516.5469 - potassium_pred_mae: 75.8484 - soil_type_pred_loss: 6.6504 - soil_type_pred_sparse_categorical_accuracy: 0.3492 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 11437.8428 - val_nitrogen_pred_loss: 351.6647 - val_nitrogen_pred_mae: 15.0875 - val_organic_matter_pred_loss: 13.0322 - val_organic_matter_pred_mae: 3.1589 - val_ph_pred_loss: 1.2426 - val_ph_pred_mae: 0.9156 - val_phosphorus_pred_loss: 587.9656 - val_phosphorus_pred_mae: 19.2712 - val_potassium_pred_loss: 10551.5059 - val_potassium_pred_mae: 74.6715 - val_soil_type_pred_loss: 3.0494 - val_soil_typ

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - health_score_pred_loss: 0.1065 - health_score_pred_mae: 0.3109 - loss: 11110.2549 - nitrogen_pred_loss: 428.6425 - nitrogen_pred_mae: 16.2472 - organic_matter_pred_loss: 14.5660 - organic_matter_pred_mae: 3.2048 - ph_pred_loss: 11.7376 - ph_pred_mae: 2.6943 - phosphorus_pred_loss: 622.2665 - phosphorus_pred_mae: 19.6677 - potassium_pred_loss: 10026.1689 - potassium_pred_mae: 74.6808 - soil_type_pred_loss: 4.4414 - soil_type_pred_sparse_categorical_accuracy: 0.4161 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 10501.0811 - val_nitrogen_pred_loss: 331.5369 - val_nitrogen_pred_mae: 14.8189 - val_organic_matter_pred_loss: 10.3413 - val_organic_matter_pred_mae: 2.7978 - val_ph_pred_loss: 2.0059 - val_ph_pred_mae: 1.1400 - val_phosphorus_pred_loss: 567.6130 - val_phosphorus_pred_mae: 18.7615 - val_potassium_pred_loss: 9700.2529 - val_potassium_pred_mae: 71.2555 - val_soil_type_pred_loss: 1.8222 - val_soil_type

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - health_score_pred_loss: 0.1069 - health_score_pred_mae: 0.3113 - loss: 11389.2744 - nitrogen_pred_loss: 416.8563 - nitrogen_pred_mae: 16.0382 - organic_matter_pred_loss: 15.0177 - organic_matter_pred_mae: 3.2611 - ph_pred_loss: 7.4613 - ph_pred_mae: 2.1606 - phosphorus_pred_loss: 615.2213 - phosphorus_pred_mae: 19.4856 - potassium_pred_loss: 10329.3408 - potassium_pred_mae: 75.4657 - soil_type_pred_loss: 3.4435 - soil_type_pred_sparse_categorical_accuracy: 0.4412 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 9975.2871 - val_nitrogen_pred_loss: 308.9410 - val_nitrogen_pred_mae: 14.4749 - val_organic_matter_pred_loss: 11.7278 - val_organic_matter_pred_mae: 2.9671 - val_ph_pred_loss: 0.6385 - val_ph_pred_mae: 0.6337 - val_phosphorus_pred_loss: 517.7794 - val_phosphorus_pred_mae: 18.5108 - val_potassium_pred_loss: 9208.3828 - val_potassium_pred_mae: 72.5688 - val_soil_type_pred_loss: 1.2090 - val_soil_type_p

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - health_score_pred_loss: 0.1066 - health_score_pred_mae: 0.3110 - loss: 10916.7930 - nitrogen_pred_loss: 413.3047 - nitrogen_pred_mae: 15.7142 - organic_matter_pred_loss: 14.6758 - organic_matter_pred_mae: 3.1890 - ph_pred_loss: 4.6871 - ph_pred_mae: 1.7362 - phosphorus_pred_loss: 598.0640 - phosphorus_pred_mae: 19.5533 - potassium_pred_loss: 9881.6299 - potassium_pred_mae: 74.6363 - soil_type_pred_loss: 2.8123 - soil_type_pred_sparse_categorical_accuracy: 0.4335 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 9538.9297 - val_nitrogen_pred_loss: 329.7290 - val_nitrogen_pred_mae: 14.9729 - val_organic_matter_pred_loss: 11.7609 - val_organic_matter_pred_mae: 2.9788 - val_ph_pred_loss: 1.5697 - val_ph_pred_mae: 1.0164 - val_phosphorus_pred_loss: 526.8428 - val_phosphorus_pred_mae: 18.5168 - val_potassium_pred_loss: 8788.5742 - val_potassium_pred_mae: 69.9178 - val_soil_type_pred_loss: 1.1687 - val_soil_type_pr

150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - health_score_pred_loss: 0.1066 - health_score_pred_mae: 0.3111 - loss: 11361.6396 - nitrogen_pred_loss: 399.8287 - nitrogen_pred_mae: 15.6420 - organic_matter_pred_loss: 14.0596 - organic_matter_pred_mae: 3.0939 - ph_pred_loss: 3.9163 - ph_pred_mae: 1.5939 - phosphorus_pred_loss: 596.0806 - phosphorus_pred_mae: 19.6033 - potassium_pred_loss: 10344.2295 - potassium_pred_mae: 75.5589 - soil_type_pred_loss: 2.2082 - soil_type_pred_sparse_categorical_accuracy: 0.4271 - val_health_score_pred_loss: 0.1004 - val_health_score_pred_mae: 0.3011 - val_loss: 9204.0898 - val_nitrogen_pred_loss: 318.0108 - val_nitrogen_pred_mae: 14.7605 - val_organic_matter_pred_loss: 11.8857 - val_organic_matter_pred_mae: 2.9837 - val_ph_pred_loss: 1.7387 - val_ph_pred_mae: 1.0680 - val_phosphorus_pred_loss: 521.6646 - val_phosphorus_pred_mae: 18.6013 - val_potassium_pred_loss: 8415.3027 - val_potassium_pred_mae: 69.7412 - val_soil_type_pred_loss: 0.9763 - val_soil_type_p

✅ Training completed in 0:00:43.264910
💾 Final model saved to soilfusion_final_model_20251004_163401.h5

🧪 Testing model...
📊 Test Results:
   pH: 7.69
   Organic Matter: 0.00%
   Nitrogen: 36.0 ppm
   Phosphorus: 50.1 ppm
   Potassium: 212.8 ppm
   Soil Type: silty (confidence: 0.548)
   Health Score: 100.0

🎯 SOILFUSION TRAINING COMPLETED!
✅ Model trained on 3000 samples
✅ Memory optimized for Colab
✅ Production-ready model saved
🎉 Training completed successfully!


In [1]:
# Install packages
!pip install tensorflow pandas numpy matplotlib scikit-learn

# The complete training script will be here
# (Copy the entire content from SOILFUSION_COLAB_COMPLETE.py)